## **Orangeshop Project**
### **Traitement des données sur les boutiques**

In [2]:
# Import des librairies
import pandas as pd
import numpy as np
import re

In [3]:
#Lecture du fichier csv sur les boutiques oranges
df = pd.read_csv("/home/etienne/Documents/etienne/Documents/VDE Python/EData-P1-boutique-free-plus-proche/free_shop.csv", sep=';')
df.head()

,longitude,latitude,shopDescription,sunusng,ipom,weiwei
0,-1.51815,43.49507,[Orange] 64 Anglet (Avenue Belle Marion),5,7,10
1,-1.42988,46.66976,[Orange] 85 La Roche-sur-Yon (15 Rue Georges C...,3,3,2
2,-1.43024,46.69141,[Orange] 85 La Roche-sur-Yon (CC Les Flaneries),5,9,5
3,-1.54907,47.16152,"[Orange] 44 Rezé (10 rond-point Corbinerie ,CC...",8,4,8
4,-1.46916,47.18722,[Orange] 44 Basse-Goulaine (CC Leclerc Pôle Sud),0,4,5


In [ ]:
#Extraction du departement de la description
df['departement'] = df['shopDescription'].str.extract(r'\[Orange\]\s+(\d+)')
df['departement'] = df['departement'].astype('Int64')
df['departement']

# Extraction de la ville
df['ville'] = df['shopDescription'].str.extract(r'\[Orange\]\s+\d+\s+([^\(]+)')
df['ville'] = df['ville'].str.strip()
df['ville']

# Extraction de l'adresse
df['adresse'] = df['shopDescription'].str.extract(r'\(([^\)]+)\)')
df['adresse'] = df['adresse'].str.strip()
df['adresse']

# nettoyage de la description et son renomage
df['shopDescription_clean'] = df['shopDescription'].str.replace(r'^\[Orange\]\s+\d+\s+', '', regex=True)
df['shopDescription_clean']


import time
from geopy.geocoders import Nominatim
# Initialiser le géocodeur
geolocator = Nominatim(user_agent="mon_application")
df["code_postal"], df["departement"] = None, None  # Initialisation

for index, row in df.iterrows():
    time.sleep(1)  # Pause de 1 seconde pour éviter d’être bloqué
    location = geolocator.reverse((row["latitude"], row["longitude"]), language="fr")
    if location and location.raw.get("address"):
        adresse = location.raw["address"]
        df.at[index, "code_postal"] = adresse.get("postcode", "Non trouvé")
# On garde les colonnes sur les boutqiues
df.drop(columns=["samsung", "iphone", "huawei"], inplace=True)
boutique=df.drop(["shopDescription", "shopDescription_clean"], axis=1)
boutique1 = pd.DataFrame(columns=["id_boutique", "nom", "id_gestionnaire_stock", "code_postal", "departement", "num_telephone", "email"])

# Définir les types après la création
boutique1 = boutique1.astype({
    "id_boutique": int,
    "nom": str,
    "id_gestionnaire_stock": int,
    "code_postal": int,
    "departement": str,
    "num_telephone": str,
    "email": str
})
boutique["id_boutique"]=boutique.index

boutiqueV=pd.concat([boutique, boutique1]).copy()

#Sauvegarde de la table boutique
boutique.to_csv("tb_boutique.csv", index=False)


## **Produit**
### **Telechargement des produits sur kaggle et leur traitement**

In [ ]:
import numpy as np 
import pandas as pd #

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import kagglehub

# Download latest version
path = kagglehub.dataset_download("shubhambathwal/flipkart-mobile-dataset")

print("Path to dataset files:", path)
print("Path to dataset files:", path)

In [ ]:
#Lecture du fichier sur les produits
df = pd.read_csv("/home/etienne/.cache/kagglehub/datasets/shubhambathwal/flipkart-mobile-dataset/versions/4/Flipkart Mobile - 2.csv")
df.head()

In [ ]:
# Renommons les colonnes les colonnes

rename_dict = {
    "brand": "nom",
    "model": "modele",
    "base_color": "couleur",
    "processor": "processeur",
    "screen_size": "taille_écran",
    "ROM": "capacite",
    "RAM": "ram",
    "display_size": "taille_affichage",
    "num_rear_camera": "nombre_caméras_arrière",
    "num_front_camera": "nombre_caméras_avant",
    "battery_capacity": "capacité_batterie",
    "ratings": "evaluations",
    "num_of_ratings": "nombre_évaluations",
    "sales_price": "prix",
    "discount_percent": "pourcentage_réduction",
    "sales": "ventes"
}

# Renommer les colonnes
df.rename(columns=rename_dict, inplace=True)




### **Scrappons les les images produis**

In [ ]:

import os
import requests
from serpapi import Client
from urllib.parse import quote, urlsplit, urlunsplit
from time import sleep
from decouple import config

df_unique=df.drop_duplicates(subset ="modele", keep = 'first').copy(deep=True)
# === Configuration ===
API_KEY = config("API_KEY")
client = Client(api_key=API_KEY)



CSV_INPUT = "telephones.csv"  # Fichier d'entrée avec colonnes 'nom' et 'modele'
CSV_OUTPUT = "telephones_avec_images.csv"
DOSSIER_IMAGES = "images_telephones"
os.makedirs(DOSSIER_IMAGES, exist_ok=True)


# === Fonction : nettoyer l'URL pour éviter erreurs 400 ===
def clean_url(url):
   try:
       parts = urlsplit(url)
       path = quote(parts.path, safe="/")
       return urlunsplit((parts.scheme, parts.netloc, path, parts.query, parts.fragment))
   except Exception:
       return url


# === Fonction : rechercher l'image avec SerpAPI ===
def get_image_url_serpapi(marque, modele):
   try:
       query = f"{marque} {modele} smartphone"
       results = client.search({
           "engine": "google_images",
           "q": query,
           "tbm": "isch"
       })
       images = results.get("images_results", [])
       if images:
           return clean_url(images[0]["original"])
   except Exception as e:
       print(f"[ERREUR API] {marque} {modele} : {e}")
   return None


# === Fonction : télécharger une image localement ===
def telecharger_image(url, filename):
   headers = {"User-Agent": "Mozilla/5.0"}
   try:
       r = requests.get(url, headers=headers, stream=True, timeout=10)
       if r.status_code == 200:
           path = os.path.join(DOSSIER_IMAGES, filename)
           with open(path, 'wb') as f:
               for chunk in r.iter_content(1024):
                   f.write(chunk)
           return True
   except Exception as e:
       print(f"[ERREUR DOWNLOAD] {url} : {e}")
   return False


# === Traitement ===


# Récupération des images (limite à 100 requêtes/jour !)
images = []
for i, row in df_unique.iterrows():
   nom = row.get("nom", "")
   modele = row.get("modele", "")
   print(f"[{i+1}/{len(df_unique)}] Recherche image pour {nom} {modele}...")
   url = get_image_url_serpapi(nom, modele)
   images.append(url)
   sleep(1.5)  # Petite pause pour éviter surcharge ou blocage


df_unique["image_url"] = images


# Téléchargement des images accessibles
for i, row in df_unique.iterrows():
   url = row["image_url"]
   nom = row.get("nom", "phone")
   modele = row.get("modele", "")
   if pd.notna(url):
       filename = f"{i}_{nom}_{modele}.jpg".replace(" ", "_")
       print(f"Téléchargement de {filename}")
       telecharger_image(url, filename)


# Sauvegarde du CSV enrichi
df_unique.to_csv(CSV_OUTPUT, index=False)
print(f"\n✅ Script terminé. Données enregistrées dans '{CSV_OUTPUT}' et images dans '{DOSSIER_IMAGES}/'")


#sans_na = df_unique[df_unique['image_url'].notna()]

df=df_unique.copy(deep=True)




### **Transformation et enregistrement des données produit,marque, modele, stock**
#### **Transformantion des données**

In [ ]:

import pandas as pd
df=pd.read_csv("/home/etienne/Documents/etienne/Documents/VDE Python/EData-P1-boutique-free-plus-proche/free_api/mobile_locator/df.csv", sep=",")
# Création des ID de marque
df["id_marque"] = df.groupby("nom").ngroup()

# Création de la table tb_marque
tb_marque = df[["id_marque", "nom"]].drop_duplicates().copy()


tb_marque.rename(columns={"nom": "marque"}, inplace=True)

# Création de la table tb_modele
df["id_modele"] = df.groupby(["modele", "id_marque"]).ngroup()
tb_modele = df[["id_modele", "id_marque", "modele"]].drop_duplicates().copy()

# Création de la table tb_produit

df["id_produit"] = df.index
df["produit"]= df[["nom", "modele", "capacite"]].astype(str).agg(" ".join, axis=1)
tb_produit = df[["id_produit", "produit", "id_modele", "prix", "couleur", "capacite", "ram", "image"]].copy()
tb_produit.rename(columns={"produit": "nom"}, inplace=True)


# Création de la table tb_stock (exemple de structure vide pour l'instant)
tb_stock = pd.DataFrame(columns=["id_boutique", "id_produit", "quantite"])

# Exemple pour remplir tb_stock avec des données fictives si tu veux tester
# Supposons que chaque produit est en stock dans 1 boutique avec une quantité aléatoire
import numpy as np
tb_stock["id_boutique"] = np.random.randint(1, 5, size=len(tb_produit))
tb_stock["id_produit"] = tb_produit["id_produit"].values
tb_stock["quantite"] = np.random.randint(1, 100, size=len(tb_produit))

